Information retrieval is one of the highly used applications of NLP and it is
quite tricky. The meaning of the words or sentences not only depends on
the exact words used but also on the context and meaning. Two sentences
may be of completely different words but can convey the same meaning.
We should be able to capture that as well

An Information retreival system allows user to efficiently seach documents and retrieve information based on the seach text

There are multiple ways to do Information retrieval. But we will see how to
do it using word embeddings, which is very effective since it takes context
also into consideration.

### 1. Import the library

In [4]:
import pandas as pd
import numpy as np
import gensim
from gensim.models import Word2Vec
import nltk
import itertools
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.tokenize.toktok import ToktokTokenizer
import scipy
import re

stopword_list = stopwords.words('english')

### 2. Create/Import the document

In [6]:
Doc1 = ["With the Union cabinet approving the amendments to the Motor Vehicles Act, 2016, those caught for drunken driving will have to have really deep pockets, as the fine payable in courthas been enhanced to Rs 10,000 for first-time offenders." ]
Doc2 = ["Natural language processing (NLP) is an area of computer science and artificial intelligence concerned with the interactions between computers and human (natural) languages, in particular how to program computers to process and analyze large amounts of natural language data."]
Doc3 = ["He points out that public transport is very good in Mumbai and New Delhi, where there is a good network of suburban and metro rail systems."]
Doc4 = ["But the man behind the wickets at the other end was watching just as keenly. With an affirmative nod from Dhoni,India captain Rohit Sharma promptly asked for a review. Sure enough, the ball would have clipped the top of middle and leg."]

In [9]:
# put all doc in one list

fin = Doc1 + Doc2 + Doc3 + Doc4

In [10]:
fin

['With the Union cabinet approving the amendments to the Motor Vehicles Act, 2016, those caught for drunken driving will have to have really deep pockets, as the fine payable in courthas been enhanced to Rs 10,000 for first-time offenders.',
 'Natural language processing (NLP) is an area of computer science and artificial intelligence concerned with the interactions between computers and human (natural) languages, in particular how to program computers to process and analyze large amounts of natural language data.',
 'He points out that public transport is very good in Mumbai and New Delhi, where there is a good network of suburban and metro rail systems.',
 'But the man behind the wickets at the other end was watching just as keenly. With an affirmative nod from Dhoni,India captain Rohit Sharma promptly asked for a review. Sure enough, the ball would have clipped the top of middle and leg.']

### 3. Load the word2vec model

In [11]:
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

### 4. Build the IR system

In [19]:
# Preprocess the data

def remove_stopwords(text, is_lower_case = False):
    pattern = r'[^a-zA-z0-9\s]'
    text = re.sub(pattern,"", "".join(text))
    tokens = word_tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_token = [token for token in tokens if token not in stopword_list]
    else:
        filtered_token = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = " ".join(filtered_token)
    return filtered_text

In [24]:
# Function to get the embedding vector for n dimension, we have used "300"

def get_embedding(word):
    if word in model.key_to_index:
        return model[word]
    else:
        return np.zeros(300)
    

In [32]:
# Getting the average vector for every document
out_dict = {}
for sen in fin:
    average_vector = (np.mean(np.array([get_embedding(x) for x in nltk.word_tokenize(remove_stopwords(sen))]), axis=0))
    dict = { sen : (average_vector) }
    out_dict.update(dict)
    
# Function to calculate the similarity between query vector and the average vector in the document
def get_sim(query_embedding, average_vector_doc):
    sim = [(1 - scipy.spatial.distance.cosine(query_embedding, average_vector_doc))]
    return sim

# Rank all the documents based on the similarity to get relevant docs


def ranked_doc(query):
    query_words = (np.mean(np.array([get_embedding(x) for x in 
                                      nltk.word_tokenize(remove_stopwords(query.lower()))], dtype=float), axis=0))
    rank = []
    for k, v in out_dict.items():
        rank.append((k, get_sim(query_words, v)))
    rank = sorted(rank, key=lambda t:t[1], reverse = True)
    print('Rank Document:')
    return rank

### 5. Result and application

In [36]:
ranked_doc('cricket')

Rank Document:


[('But the man behind the wickets at the other end was watching just as keenly. With an affirmative nod from Dhoni,India captain Rohit Sharma promptly asked for a review. Sure enough, the ball would have clipped the top of middle and leg.',
  [0.35923325370062154]),
 ('He points out that public transport is very good in Mumbai and New Delhi, where there is a good network of suburban and metro rail systems.',
  [0.23973446930269127]),
 ('Natural language processing (NLP) is an area of computer science and artificial intelligence concerned with the interactions between computers and human (natural) languages, in particular how to program computers to process and analyze large amounts of natural language data.',
  [0.17995061678671642]),
 ('With the Union cabinet approving the amendments to the Motor Vehicles Act, 2016, those caught for drunken driving will have to have really deep pockets, as the fine payable in courthas been enhanced to Rs 10,000 for first-time offenders.',
  [0.1794264

In [37]:
ranked_doc("driving")

Rank Document:


[('With the Union cabinet approving the amendments to the Motor Vehicles Act, 2016, those caught for drunken driving will have to have really deep pockets, as the fine payable in courthas been enhanced to Rs 10,000 for first-time offenders.',
  [0.3598919009839452]),
 ('But the man behind the wickets at the other end was watching just as keenly. With an affirmative nod from Dhoni,India captain Rohit Sharma promptly asked for a review. Sure enough, the ball would have clipped the top of middle and leg.',
  [0.2037214882585675]),
 ('He points out that public transport is very good in Mumbai and New Delhi, where there is a good network of suburban and metro rail systems.',
  [0.1706653724240128]),
 ('Natural language processing (NLP) is an area of computer science and artificial intelligence concerned with the interactions between computers and human (natural) languages, in particular how to program computers to process and analyze large amounts of natural language data.',
  [0.0887230840